# reader

> Necessary scripts to read orbits from different formats

In [1]:
#| default_exp reader

In [2]:
#| export
import h5py
from scipy.io import loadmat
import numpy as np

In [ ]:
from unittest.mock import patch, MagicMock

## Loading Data

This function loads orbit data from .mat or .h5

In [3]:
#| export
def load_orbit_data(file_path, variable_name=None, dataset_path=None):
    """
    Load orbit data from MATLAB .mat files or HDF5 .h5 files.

    Parameters:
    - file_path: str, the path to the .mat or .h5 file.
    - variable_name: str, optional, the name of the variable in the .mat file.
    - dataset_path: str, optional, the path to the dataset in the .h5 file.

    Returns:
    - data: The loaded orbit data.
    """
    if file_path.endswith('.mat'):
        if variable_name is None:
            raise ValueError("variable_name must be provided for .mat files")
        mat = loadmat(file_path)
        if variable_name in mat:
            data = mat[variable_name]
        else:
            raise ValueError(f"{variable_name} not found in {file_path}")
    
    elif file_path.endswith('.h5'):
        with h5py.File(file_path, 'r') as file:
            if dataset_path:
                if dataset_path in file:
                    data = np.array(file[dataset_path])
                else:
                    raise ValueError(f"{dataset_path} not found in {file_path}")
    
    else:
        raise ValueError("Unsupported file format. Please provide a .mat or .h5 file.")
    
    return data

In [4]:
#| test "load_orbit_data with .mat file"
mock_mat_data = {'Xarray': np.array([1, 2, 3])}
mock_h5_data = np.array([4, 5, 6])

# Test for load_orbit_data with .mat file
with patch('__main__.loadmat', return_value=mock_mat_data) as mock_loadmat:
    result = load_orbit_data('test_data.mat', variable_name='Xarray')
    assert (result == mock_mat_data['Xarray']).all(), "MAT file loading failed or data mismatch"
    mock_loadmat.assert_called_once_with('test_data.mat')

# Test for load_orbit_data with .h5 file
with patch('__main__.h5py.File') as mock_h5py:
    mock_file = MagicMock()
    mock_file.__enter__.return_value = {'/files/PERIODIC ORBITS': mock_h5_data}
    mock_h5py.return_value = mock_file
    result = load_orbit_data('test_data.h5', dataset_path='/files/PERIODIC ORBITS')
    assert (result == mock_h5_data).all(), "H5 file loading failed or data mismatch"

### Functions to return data directly

In [5]:
#| export
def get_example_h5_data():
    hdf5_file_path = r"C:\Users\alvar\Desktop\Orbit\Data\em_orbits.h5"
    dataset_path = '/files/PERIODIC ORBITS'
    data_h5 = load_orbit_data(hdf5_file_path, dataset_path=dataset_path)
    no_time_data = data_h5[:, 1:, :]
    return no_time_data

In [6]:
#| export
def get_example_mat_data():
    matlab_file_path = r"C:\Users\alvar\Desktop\Orbit\Data\L2_S_200_EM_CR3BP.mat"
    variable_name = 'Xarray'
    data_mat = load_orbit_data(matlab_file_path, variable_name=variable_name)
    data_mat= np.transpose(data_mat, (2, 1, 0))
    return data_mat

In [7]:
data_h5 = get_example_h5_data()
print(data_h5.shape)

(36071, 6, 7500)


- Number of orbits: 36071
- Time instants: 7500

In [8]:
data_mat= get_example_mat_data()
print(data_mat.shape)

(200, 6, 300)


- Number of orbits: 200
- Time instants: 300

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()